In [1]:
from sklearn.metrics import classification_report
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.models import Model as KerasModel
from tensorflow.python.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.python.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from pprint import pprint
random.seed(32)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2" # Choose GPU NUMBERS [0, 1, 2, 3]
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

In [2]:
def generate_generator_multiple(generator,dir1, dir2,dir3,dir4,dir5, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'binary',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)   
    genX2 = generator.flow_from_directory(dir2,
                                          target_size = (img_height,img_width),
                                          class_mode = 'binary',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)
    genX3 = generator.flow_from_directory(dir3,
                                          target_size = (img_height,img_width),
                                          class_mode = 'binary',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)
    genX4 = generator.flow_from_directory(dir4,
                                          target_size = (img_height,img_width),
                                          class_mode = 'binary',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)
    genX5 = generator.flow_from_directory(dir5,
                                          target_size = (img_height,img_width),
                                          class_mode = 'binary',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)
#     for i in range(0,len(genX1.filepaths)+len(genX2.filepaths)):
    i=0
    while(True):
        if i<len(genX1.filepaths):
            X1i = genX1.next()
            yield X1i[0], X1i[1]
            
        if i<len(genX2.filepaths):
            X2i = genX2.next()
            yield X2i[0], X2i[1] 
            
        if i<len(genX3.filepaths):
            X3i = genX3.next()
            yield X3i[0], X3i[1]
            
        if i<len(genX4.filepaths):
            X4i = genX4.next()
            yield X4i[0], X4i[1] 
            
        if i<len(genX5.filepaths):
            X5i = genX5.next()
            yield X5i[0], X5i[1]
        i+=1

In [3]:
batch_size=64
img_height=128
img_width=128
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator=generate_generator_multiple(generator=train_datagen,
                                           dir1='/media/data1/sha/CLRNet/DeepFake/train',
                                           dir2='/media/data1/sha/CLRNet/FaceSwap/train',
                                           dir3='/media/data1/sha/CLRNet/Face2Face/train',
                                           dir4='/media/data1/sha/CLRNet/NeuralTextures/train',
                                           dir5='/media/data1/sha/CLRNet/DeepFakeDetection/train',
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)    
validation_generator=generate_generator_multiple(validation_datagen,
                                          dir1='/media/data1/sha/CLRNet/DeepFake/val',
                                          dir2='/media/data1/sha/CLRNet/FaceSwap/val',
                                          dir3='/media/data1/sha/CLRNet/Face2Face/val',
                                          dir4='/media/data1/sha/CLRNet/NeuralTextures/val',
                                          dir5='/media/data1/sha/CLRNet/DeepFakeDetection/val',
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_width) 

In [4]:
x=train_generator.__next__()
print(x[1].shape)
pprint(x[1])
plt.imshow(x[0][1])
plt.title(x[1][1])

In [5]:
x=validation_generator.__next__()
print(x[1].shape)
pprint(x[1])
plt.imshow(x[0][1])
plt.title(x[1][1])

In [4]:
def get_pred(predictions):
    pred=[]
    for p in predictions:
        if p<0.50:
             pred.append(0)
        else:
            pred.append(1)
    return pred

IMGWIDTH = 128

class Classifier:
    def __init__(self):
        self.model = 0
    
    def predict(self, x):
        return self.model.predict(x)
    
    def fit(self, x, y):
        return self.model.train_on_batch(x, y)
    
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)
    
    def load(self, path):
        self.model.load_weights(path)


class MesoInception4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
    
    def InceptionLayer(self, a, b, c, d):
        def func(x):
            x1 = Conv2D(a, (1, 1), padding='same', activation='relu')(x)
            
            x2 = Conv2D(b, (1, 1), padding='same', activation='relu')(x)
            x2 = Conv2D(b, (3, 3), padding='same', activation='relu')(x2)
            
            x3 = Conv2D(c, (1, 1), padding='same', activation='relu')(x)
            x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides = 1, padding='same', activation='relu')(x3)
            
            x4 = Conv2D(d, (1, 1), padding='same', activation='relu')(x)
            x4 = Conv2D(d, (3, 3), dilation_rate = 3, strides = 1, padding='same', activation='relu')(x4)

            y = Concatenate(axis = -1)([x1, x2, x3, x4])
            
            return y
        return func
    
    def init_model(self):
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))
        
        x1 = self.InceptionLayer(1, 4, 4, 2)(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = self.InceptionLayer(2, 4, 4, 2)(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)        
        
        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return KerasModel(inputs = x, outputs = y)

In [7]:
model= MesoInception4().model
model_checkpoint_callback = ModelCheckpoint(
    filepath="MI4_ALL_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
csv_logger = CSVLogger("MI4_ALL_train.csv", append=True, separator=',')
print_summary(model, line_length=115, positions=None, print_fn=None)

In [8]:
total_train=520000/8 #65,000
total_val=84000/8   #12,500
model.fit_generator(train_generator,
                    verbose=1,
                    steps_per_epoch=int(total_train/batch_size),
                    epochs=50,
                    validation_data=validation_generator,
                    validation_steps = int(total_val/batch_size),
                    callbacks=[model_checkpoint_callback,csv_logger])
model.save_weights("MI4_ALL_E50.h5")

In [9]:
# model= MesoInception4()
# model.load("MI4_ALL_Ebest.h5")
# model=model.model
# print_summary(model, line_length=115, positions=None, print_fn=None)

# # model = load_model('ResNet50(bk)_19-12-11-12_38-150-1.00.hdf5')
# # adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# # model.compile(loss='binary_crossentropy',
# #                   optimizer=adam_fine,
# #                   metrics=['accuracy'])

# DeepFakeDetection Test 

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_ALL_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakeDetection/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFake Test 

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_ALL_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFake/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# FaceSwap Test 

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_ALL_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/FaceSwap/test',# this is the target directory
        target_size=(128,128),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# Face2Face Test 

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_ALL_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/Face2Face/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# NeuralTextures Test 

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_ALL_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/NeuralTextures/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFakewild Test 

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_ALL_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakewild/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DFDC Test 

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("MI4_ALL_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DFDC/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# New DeepFake wild Test

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
model= MesoInception4()
model.load("best_models/MI4_ALL_Ebest.h5")
model=model.model
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DFP_new/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='binary',
        shuffle=False,
        seed=32) 
predictions=model.predict_generator(test_generator,verbose=1)
predicted_classes = get_pred(predictions)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)